In [1]:
#hide
#skip
%config Completer.use_jedi = False
%config IPCompleter.greedy=True
# upgrade fastrl on colab
! [ -e /content ] && pip install -Uqq fastrl['dev'] pyvirtualdisplay && \
                     apt-get install -y xvfb python-opengl > /dev/null 2>&1 
# NOTE: IF YOU SEE VERSION ERRORS, IT IS SAFE TO IGNORE THEM. COLAB IS BEHIND IN SOME OF THE PACKAGE VERSIONS

In [114]:
# hide
from fastcore.imports import in_colab
# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbverbose.showdoc import *
    from nbdev.imports import *
    if not os.environ.get("IN_TEST", None):
        assert IN_NOTEBOOK
        assert not IN_COLAB
        assert IN_IPYTHON
else:
    # Virutual display is needed for colab
    from pyvirtualdisplay import Display
    display = Display(visible=0, size=(400, 300))
    display.start()

In [115]:
# default_exp fastai.data.pipes

In [116]:
# export
# Python native modules
import os
from inspect import isfunction,ismethod
from typing import *
# Third party libs
from fastcore.all import *
from fastai.torch_basics import *
# from torch.utils.data.dataloader import DataLoader as OrgDataLoader
import torchdata.datapipes as dp
from torch.utils.data.dataloader_experimental import DataLoader2
from fastai.data.transforms import *
# Local modules
from fastrl.fastai.loop import *
from fastrl.fastai.data.load import *

# Basic DataPipes
> Basic datapipes for work with fastrl core API.

In [5]:
#For example, so not exported

from fastai.vision.core import *
from fastai.vision.data import *
from fastai.data.external import *

untar_data(URLs.MNIST_TINY)

Path('/home/fastrl_user/.fastai/data/mnist_tiny')

Load the mnist csv...

In [47]:
pipe = dp.iter.IterableWrapper([str(untar_data(URLs.MNIST_TINY)/'labels.csv')]) # FileOpener really should support Path as well as str
pipe = dp.iter.FileOpener(pipe, mode="b")
pipe = dp.iter.CSVParser(pipe,skip_lines=1)

class AddIdx():
    def __init__(self): self.idx=0
    def __call__(self,file):
        try:     return (self.idx,file)
        finally: self.idx+=1

base_pipe = dp.map.IterToMapConverter(pipe,key_value_fn=AddIdx())
pipe = dp.map.IterToMapConverter(pipe,key_value_fn=AddIdx())
pipe[5],len(base_pipe)

/home/fastrl_user/src/torchdata/torchdata/datapipes/map/util/utils.py:78: UserWarning: Data from prior DataPipe are loaded to get length ofIterToMapConverter before execution of the pipeline.Please consider removing len().
  "Data from prior DataPipe are loaded to get length of"


(['train/3/7745.png', '3'], 1408)

In [60]:
# [pipe[i] for i in range(len(pipe))]

In [ ]:
pipe

Now that we have the csv converted into a map, we want to split it into a training and validation dataset...

In [ ]:
iterator=iter({1,2,3,4})
next(iterator)

1

In [120]:
import warnings

from collections import deque
from typing import Any, Callable, Iterator, List, Optional, Set, Sized, Tuple, TypeVar, Deque

from torch.utils.data import IterDataPipe, functional_datapipe
from torch.utils.data.datapipes.utils.common import check_lambda_fn
from torch.utils.data._utils.serialization import serialize_fn, deserialize_fn


T_co = TypeVar("T_co", covariant=True)

from collections import deque
from collections.abc import Hashable


class _ChildMapDataPipe(dp.map.MapDataPipe):
    r"""
    Map Datapipe that is a child of a main DataPipe. The instance of this class
    will pass its instance_id to get the next value from its main DataPipe.

    Args:
        main_datapipe: Main DataPipe with a method 'get_next_element_by_instance(instance_id)'
        instance_id: integer identifier of this instance
    """
    def __init__(self, main_datapipe, instance_id: Hashable):
        
        required_attrs = ["get_next_element_by_instance", "is_instance_started", "is_every_instance_exhausted", "reset"]
        required_ops = [getattr(main_datapipe, attr) for attr in required_attrs]
        if any(not callable(op) for op in required_ops):
            raise NotImplementedError(f"Main Datapipe must have methods {required_attrs} implemented.")
        self.main_datapipe = main_datapipe
        self.instance_id = instance_id

    def __iter__(self):
        # if self.main_datapipe.is_instance_started(self.instance_id):  # Only reset if the DataPipe started to read
        #     if not self.main_datapipe.is_every_instance_exhausted():
        #         warnings.warn("Some child DataPipes are not exhausted when __iter__ is called. We are resetting "
        #                       "the buffer and each child DataPipe will read from the start again.", UserWarning)
        #     self.main_datapipe.reset()
        
        # These is no concept of exhaustion of the 'main_datapipe'. We only need
        # to run through it once,, then use the cached indexes for querying.
        return self.get_generator_by_instance(self.instance_id)

    def __len__(self):
        # Need to be careful here,  the len of `_ChildMapDataPipe` will be <= len(self.main_datapipe)
        return len(self.main_datapipe.get_instance_buffer(self.instance_id))

    def get_generator_by_instance(self, instance_id: Hashable):
        yield from self.main_datapipe.get_next_element_by_instance(self.instance_id)
        
    def __getitem__(self, index):
        "Gets an item from `self.main_datapipe` in `self.instance_id`"
        return self.main_datapipe.getitem_by_instance(self.instance_id, index)


class _DemultiplexerMapDataPipe(dp.map.MapDataPipe):
    r"""
    Container to hold instance-specific information on behalf of _DemultiplexerMapDataPipe. It tracks
    the state of its child DataPipes, maintains the buffer, classifies and yields the next correct value
    as requested by the child DataPipes.
    """

    def __init__(self, datapipe: dp.map.MapDataPipe[T_co], 
                 # num_instances: int,
                 instance_keys: Hashable,
                 classifier_fn: Callable[[T_co], Optional[int]], drop_none: bool, buffer_size: int):
        self.main_datapipe = datapipe
        self._datapipe_indexer: Optional[Iterator[Any]] = None
        # self._datapipe_iterator: Optional[Iterator[Any]] = None
        self.instance_keys = instance_keys
        self.buffer_size = buffer_size
        if self.buffer_size < 0:
            warnings.warn(
                "Unlimited buffer size is set for `demux`, "
                "please be aware of OOM at random places",
                UserWarning
            )
        self.current_buffer_usage = 0
        # The child buffers will store the indexes separated into their respective
        # `_ChildMapDataPipe`'s
        self.child_index_buffers: Dict[set[T_co]] = {k:set() for k in self.instance_keys}
        self.instance_started: Dict[Hashable,bool] = {k:False for k in instance_keys}
        self.classifier_fn = classifier_fn
        self.drop_none = drop_none
        self.main_datapipe_exhausted = False
        
    def _setup_datapipe_indexer(self) -> Optional[Iterator[Any]]:
        # self._datapipe_iterator: Optional[Iterator[Any]] = None
        # Instead of _datapipe_iterator we have _datapipe_indexer
        # We need to know how to get the index from the main_datapipe. In order
        # to do this, we check if it is...
        
        # NOTE: THIS IS NOT A GOOD SOLUTION.
        
        # We cash the indexes because we want to be able to have consistent behavior 
        # when calling __getitem__ on a child pipe. 
        # What we don't want is the main_datapipe being indexed by `str` but the
        # child pipes indexing by `int`...
        if isinstance(self.main_datapipe, dp.map.SequenceWrapper):
            return range(len(self.main_datapipe))
        elif hasattr(self.main_datapipe, '_map'):
            return iter(self.main_datapipe._map)
        elif hasattr(self.main_datapipe, 'index_map'):
            return iter(self.main_datapipe.index_map)
        else:
            warnings.warn('data pipe will be indexed by len')
            return range(len(self.main_datapipe))
        
    def get_instance_buffer(self, instance_id: Hashable):
        return self.child_index_buffers[instance_id]

    def _find_next(self, instance_id: Hashable) -> T_co:
        while True:
            if self.main_datapipe_exhausted:
                raise StopIteration
            if self._datapipe_indexer is None:
                raise ValueError(
                    "_datapipe_indexer has not been set, likely because this private method is called directly "
                    "without invoking get_next_element_by_instance() first.")
            index = next(self._datapipe_indexer)
            value = self.main_datapipe[index]
            classification = self.classifier_fn(value)
            if classification is None and self.drop_none:
                continue
            if classification is None or classification not in self.instance_keys:
                raise ValueError(f"Output of the classification fn should be a key in {self.instance_keys}. " +
                                 f"{classification} is returned.")
            
            if index not in self.child_index_buffers[classification]:
                self.child_index_buffers[classification].add(index)
                self.current_buffer_usage += 1
                if self.buffer_size >= 0 and self.current_buffer_usage > self.buffer_size:
                    raise BufferError(
                        f"DemultiplexerIterDataPipe buffer overflow, buffer size {self.buffer_size} is insufficient.")

            if classification == instance_id:
                return value,index

    def getitem_by_instance(self, instance_id: Hashable, index: Hashable):
        # We need to handle the situation where the index is not currently cached.
        # In this case we still need to build the cache, while still attempting to 
        # get the value for `index`
        
        # In this case, `main_datapipe_exhausted` which means we still have some
        # of the cache to populate possibly.
        # Josiah: The main_datapipe_exhausted doesnt make sense in this context.
        if index in self.child_index_buffers[instance_id]:
            return self.main_datapipe[index]
        
        if self._datapipe_indexer is None and not self.main_datapipe_exhausted:
            self._datapipe_indexer = iter(self._setup_datapipe_indexer())
        stop = False
        self.instance_started[instance_id] = True

        while not stop:
            # We only want to iterate through the indexes once `self._datapipe_indexer` is clear
            # so that we are "gaurenteed" to go through all the indexes possible for 
            # instance_id
            if self.child_index_buffers[instance_id] and self._datapipe_indexer is None:
                return self.main_datapipe[index]
            else:
                try:
                    value,_index = self._find_next(instance_id)
                    if _index==index: return value
                except StopIteration:
                    stop = True
                    self.main_datapipe_exhausted = True
                    self._datapipe_indexer = None
    
                    
    def get_next_element_by_instance(self, instance_id: Hashable):
        # Josiah: The main_datapipe_exhausted doesnt make sense in this context.
        if self._datapipe_indexer is None and not self.main_datapipe_exhausted:
            self._datapipe_indexer = iter(self._setup_datapipe_indexer())
        stop = False
        self.instance_started[instance_id] = True
        instance_next_index = None
        
        while not stop:
            # We only want to iterate through the indexes once `self._datapipe_indexer` is clear
            # so that we are "gaurenteed" to go through all the indexes possible for 
            # instance_id
            if self.child_index_buffers[instance_id] and self._datapipe_indexer is None:
                if instance_next_index is None:
                    instance_next_indexer = iter(self.child_index_buffers[instance_id])
                index = next(instance_next_indexer)
                yield self.main_datapipe[index]
            else:
                try:
                    value,index = self._find_next(instance_id)
                    yield value
                except StopIteration:
                    stop = True
                    self.main_datapipe_exhausted = True
                    self._datapipe_indexer = None
    

    def is_instance_started(self, instance_id: Hashable) -> bool:
        return self.instance_started[instance_id]

    def is_every_instance_exhausted(self) -> bool:
        return self.main_datapipe_exhausted and all(not child_buffer for child_buffer in self.child_index_buffers)

    def reset(self):
        self._datapipe_iterator = iter(self.main_datapipe)
        self.current_buffer_usage = 0
        self.child_index_buffers: Dict[Deque[T_co]] = {k:deque() for k in self.instance_keys}
        self.instance_started: Dict[Hashable,bool] = {k:False for k in instance_keys}
        self.main_datapipe_exhausted = False

    def __getstate__(self):
        if IterDataPipe.getstate_hook is not None:
            return IterDataPipe.getstate_hook(self)

        serialized_fn_with_method = serialize_fn(self.classifier_fn)
        state = (
            self.main_datapipe,
            self.instance_keys,
            self.buffer_size,
            serialized_fn_with_method,
            self.drop_none,
        )
        return state

    def __setstate__(self, state):
        (
            self.main_datapipe,
            self.num_instances,
            self.buffer_size,
            serialized_fn_with_method,
            self.drop_none,
        ) = state
        self.classifier_fn = deserialize_fn(serialized_fn_with_method)
        self._datapipe_iterator = None
        self.current_buffer_usage = 0
        self.child_index_buffers: Dict[Deque[T_co]] = {k:deque() for k in self.instance_keys}
        self.instance_started: Dict[Hashable,bool] = {k:False for k in instance_keys}
        self.main_datapipe_exhausted = False
        

class DemultiplexerMapDataPipe(dp.map.MapDataPipe):
    def __new__(cls, datapipe: dp.map.MapDataPipe, instance_keys: List[Hashable],
                classifier_fn: Callable[[T_co], Optional[int]], drop_none: bool = False, buffer_size: int = -1):
        if not isinstance(datapipe, dp.map.MapDataPipe):
            raise TypeError(f"DemultiplexerMapDataPipe can only apply on MapDataPipe, but found {type(datapipe)}")
        if not instance_keys:
            raise ValueError(f"Expected `instance_keys` larger than 0, but {instance_keys} is found")

        check_lambda_fn(classifier_fn)

        container = _DemultiplexerMapDataPipe(datapipe, instance_keys, classifier_fn, drop_none, buffer_size)
        return [_ChildMapDataPipe(container, k) for k in instance_keys]

In [121]:
# It can also filter out any element that gets `None` from the `classifier_fn`

def train_valid_splitter(o): 
    # print(o)
    return Path(o[0]).parts[0]

dp1, dp2 = DemultiplexerMapDataPipe(pipe,instance_keys=['train','valid'], classifier_fn=train_valid_splitter, drop_none=True)
list(dp1);
# [2, 4]
# list(dp2)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:78: UserWarning: Unlimited buffer size is set for `demux`, please be aware of OOM at random places


In [122]:
list(dp1)

KeyboardInterrupt: 

In [96]:
train,valid = MultiplexerMapDataPipe(pipe)

assert 0 not in (len(train),len(valid))
assert len(train)+len(valid)==len(base_pipe)


pipe

NameError: name 'MultiplexerMapDataPipe' is not defined

In [ ]:
# hide
from fastcore.imports import in_colab

# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.export import *
    from nbdev.export2html import *
    from nbverbose.cli import *
    make_readme()
    notebook2script()